# IonQ Boostvqe demo

In this notebook, we demonstrate how to run boostvqe on IonQ emulators (similarly hardwarws with some backend modifications).

We are following on the DBQA GCI circuit synthesis notebook, which can be found following the link: https://github.com/qiboteam/boostvqe/blob/quantinuum_boostvqe-demo/notebooks/quantinuum_demo/gci_boostvqe_circuit_synthesis.ipynb

It supplies a VQE and a GCI circuits in QASM format which we will load our circuits from.

## Step 1: Load circuits

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qibo import hamiltonians
import matplotlib.pyplot as plt
from ionq_utils import *
from scipy.linalg import eigh

In [2]:
import os
os.chdir('..')
optimizer = 'sgd'
nqubits = 4
nlayer = 1
folder_path = f'results/circuit_qasm/{optimizer}_{nqubits}q_{nlayer}l_XXZ/'
vqe_circ_file = folder_path + 'vqe_circ.qasm'
gci_circ_file = folder_path + 'gci_circ.qasm'

In [3]:
vqe_circ = QuantumCircuit.from_qasm_file(vqe_circ_file)
# render_circuit_jupyter(vqe_circ)

In [4]:
gci_circ = QuantumCircuit.from_qasm_file(gci_circ_file)
# render_circuit_jupyter(gci_circ)

In [5]:
print("Circuit depth:", gci_circ.depth())
op_count = gci_circ.count_ops()
print("Circuit total gate count:", sum(op_count.values()))
print("Circuit CNOT count:", op_count['cx'])

Circuit depth: 235
Circuit total gate count: 632
Circuit CNOT count: 144


### Check analytical energy

In this step we are making sure that the correct circuits have been imported.

In [6]:
# create target hamiltonian in qiskit
ham = xxz_hamiltonian(4, 0.5)
ham_matrix = ham.to_matrix()
eigenvalues, eigenstates = eigh(ham_matrix)
target_energy = eigenvalues[0]

In [7]:
state_vec_vqe = Statevector.from_instruction(vqe_circ)
expectation_value_vqe = np.real(np.dot(state_vec_vqe.data.conj().T, np.dot(ham_matrix, state_vec_vqe.data)))
state_vec_gci = Statevector.from_instruction(gci_circ)
expectation_value_gci = np.real(np.dot(state_vec_gci.data.conj().T, np.dot(ham_matrix, state_vec_gci.data)))
print("Target energy:", target_energy)
print("VQE energy:", expectation_value_vqe)
print("GCI energy:", expectation_value_gci)

Target energy: -6.744562646538019
VQE energy: -6.309812021233839
GCI energy: -6.741347828261663


## Step 2: Connect to IonQ server

In [8]:
from qiskit_ionq import IonQProvider
from copy import deepcopy

my_api_key = "pOiUVlzriOoF2wX1kp3lIqid1OMhwXZ5"
provider = IonQProvider(my_api_key)
ionq_backend = provider.get_backend("ionq_simulator")

/Users/pethidine/Documents/GitHub/boostvqe/.conda/lib/python3.11/site-packages/qiskit_ionq/helpers.py:563: UserWarning: Unable to get qubit count for simulator: 'qubits'. Defaulting to 100.
  warnings.warn(
/Users/pethidine/Documents/GitHub/boostvqe/.conda/lib/python3.11/site-packages/qiskit_ionq/helpers.py:563: UserWarning: Unable to get qubit count for ionq_qpu: 'qubits'. Defaulting to 100.
  warnings.warn(
/Users/pethidine/Documents/GitHub/boostvqe/.conda/lib/python3.11/site-packages/qiskit_ionq/helpers.py:563: UserWarning: Unable to get qubit count for ionq_simulator: 'qubits'. Defaulting to 100.
  warnings.warn(


## Step 3: Prepare measurement circuits

In this step, we prepare a list of measurement circuits to measure the energy expectation of the Hamiltonian.
Since our example uses XXZ model, we will need to measure the circuits in 3 separate bases:
1. X
2. Y
3. Z

In [9]:
from pytket.extensions.qiskit import tk_to_qiskit
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit
from pytket.utils.operators import QubitPauliOperator

In [10]:
meas_vqe_circ_list = rotate_circuit_XYZ(vqe_circ)
meas_gci_circ_list = rotate_circuit_XYZ(gci_circ)

## Step 4: Execute circuits


In [11]:
zz = xxz_hamiltonian(4,1,select='ZZ').to_matrix()
coef = [1,1,0.5]

In [29]:
shots = 1000
observable_labels = ['XX', 'YY', 'ZZ']
vqe_expectation = 0

# Initialize a dictionary to store expectation values
results = {
    "shots": shots,
    "observables": {},
    "total_expectation_energy": 0.0
}

for i, qc_m in enumerate(meas_vqe_circ_list):
    # Execute the measurement circuit
    job = ionq_backend.run(qc_m, shots=shots)
    result = job.result()
    
    # Retrieve the probability distribution
    distribution = result.get_probabilities()
    
    # Compute the expectation value for the current observable
    expectation = sample_to_expectation(zz, distribution) * coef[i]
    
    # Accumulate the expectation value
    vqe_expectation += expectation
    
    # Store the expectation value in the results dictionary
    observable_label = observable_labels[i]
    results["observables"][observable_label] = expectation
    
    # Print the expectation value for the current observable
    print(f"{observable_label}: {expectation}")

# Store the total expectation energy
results["total_expectation_energy"] = vqe_expectation
print('Expectation energy:', vqe_expectation)

XX: -3.0812240639999997
YY: -1.96724834
ZZ: -1.272256812
Expectation energy: -6.320729216


In [30]:
import json

# Define the filename where you want to store the results
folder_path = f'results/ionq_job_results/{optimizer}_{nqubits}q_{nlayer}l_XXZ/'
os.makedirs(folder_path, exist_ok=True)
file_path = folder_path + f'{shots}shots_vqe_results.json'
# Open the file in write mode and dump the results dictionary
with open(file_path, 'w') as f:
    json.dump(results, f, indent=4)

print(f"VQE results successfully saved to {file_path}")

VQE results successfully saved to results/ionq_job_results/sgd_4q_1l_XXZ/1000shots_vqe_results.json


In [31]:
shots = 1000
observable_labels = ['XX', 'YY', 'ZZ']
gci_expectation = 0

# Initialize a dictionary to store expectation values
results = {
    "shots": shots,
    "observables": {},
    "total_expectation_energy": 0.0
}

for i, qc_m in enumerate(meas_gci_circ_list):
    # Execute the measurement circuit
    job = ionq_backend.run(qc_m, shots=shots)
    result = job.result()
    
    # Retrieve the probability distribution
    distribution = result.get_probabilities()
    
    # Compute the expectation value for the current observable
    expectation = sample_to_expectation(zz, distribution) * coef[i]
    
    # Accumulate the expectation value
    gci_expectation += expectation
    
    # Store the expectation value in the results dictionary
    observable_label = observable_labels[i]
    results["observables"][observable_label] = expectation
    
    # Print the expectation value for the current observable
    print(f"{observable_label}: {expectation}")

# Store the total expectation energy
results["total_expectation_energy"] = gci_expectation
print('Expectation energy:', gci_expectation)


XX: -2.7795153800000003
YY: -2.7820648959999996
ZZ: -1.180009208
Expectation energy: -6.741589484


In [32]:
# Define the filename where you want to store the results
folder_path = f'results/ionq_job_results/{optimizer}_{nqubits}q_{nlayer}l_XXZ/'
os.makedirs(folder_path, exist_ok=True)
file_path = folder_path + f'{shots}shots_gci_results.json'
# Open the file in write mode and dump the results dictionary
with open(file_path, 'w') as f:
    json.dump(results, f, indent=4)

print(f"GCI results successfully saved to {file_path}")

GCI results successfully saved to results/ionq_job_results/sgd_4q_1l_XXZ/1000shots_gci_results.json


## Step 5: Run with noise

In [33]:
shots = 1000
vqe_expectation_noise = 0

# Initialize a dictionary to store expectation values
results = {
    "shots": shots,
    "observables": {},
    "total_expectation_energy": 0.0
}

for i, qc_m in enumerate(meas_vqe_circ_list):
    # Execute the measurement circuit
    job = ionq_backend.run(qc_m, shots=shots, noise_model='aria-1')
    result = job.result()
    
    # Retrieve the probability distribution
    distribution = result.get_probabilities()
    
    # Compute the expectation value for the current observable
    expectation = sample_to_expectation(zz, distribution) * coef[i]
    
    # Accumulate the expectation value
    vqe_expectation_noise += expectation
    
    # Store the expectation value in the results dictionary
    observable_label = observable_labels[i]
    results["observables"][observable_label] = expectation
    
    # Print the expectation value for the current observable
    print(f"{observable_label}: {expectation}")

# Store the total expectation energy
results["total_expectation_energy"] = vqe_expectation_noise
print('Expectation energy:', vqe_expectation_noise)

XX: -2.996
YY: -1.884
ZZ: -1.196
Expectation energy: -6.076


In [21]:
# Define the filename where you want to store the results
folder_path = f'results/ionq_job_results/{optimizer}_{nqubits}q_{nlayer}l_XXZ/'
os.makedirs(folder_path, exist_ok=True)
file_path = folder_path + f'{shots}shots_vqe_results_noise.json'
# Open the file in write mode and dump the results dictionary
with open(file_path, 'w') as f:
    json.dump(results, f, indent=4)

print(f"VQE results successfully saved to {file_path}")

VQE results successfully saved to results/ionq_job_results/sgd_4q_1l_XXZ/1000shots_vqe_results_noise.json


In [23]:
shots = 1000
observable_labels = ['XX', 'YY', 'ZZ']
gci_expectation_noise = 0

# Initialize a dictionary to store expectation values
results = {
    "shots": shots,
    "observables": {},
    "total_expectation_energy": 0.0
}

for i, qc_m in enumerate(meas_gci_circ_list):
    # Execute the measurement circuit
    job = ionq_backend.run(qc_m, shots=shots, noise_model='aria-1')
    result = job.result()
    
    # Retrieve the probability distribution
    distribution = result.get_probabilities()
    
    # Compute the expectation value for the current observable
    expectation = sample_to_expectation(zz, distribution) * coef[i]
    
    # Accumulate the expectation value
    gci_expectation_noise += expectation
    
    # Store the expectation value in the results dictionary
    observable_label = observable_labels[i]
    results["observables"][observable_label] = expectation
    
    # Print the expectation value for the current observable
    print(f"{observable_label}: {expectation}")

# Store the total expectation energy
results["total_expectation_energy"] = gci_expectation_noise
print('Expectation energy:', gci_expectation_noise)

XX: -0.48
YY: -0.45600000000000007
ZZ: -0.254
Expectation energy: -1.19


In [24]:
# Define the filename where you want to store the results
folder_path = f'results/ionq_job_results/{optimizer}_{nqubits}q_{nlayer}l_XXZ/'
os.makedirs(folder_path, exist_ok=True)
file_path = folder_path + f'{shots}shots_gci_results_noise.json'
# Open the file in write mode and dump the results dictionary
with open(file_path, 'w') as f:
    json.dump(results, f, indent=4)

print(f"GCi results successfully saved to {file_path}")

GCi results successfully saved to results/ionq_job_results/sgd_4q_1l_XXZ/1000shots_gci_results_noise.json


## Step 6: Load results

In [39]:
import json
results_folder =  f'results/ionq_job_results/{optimizer}_{nqubits}q_{nlayer}l_XXZ/'
vqe_path = results_folder + '1000shots_vqe_results.json'
with open(vqe_path, 'r') as f:
    vqe_results = json.load(f)

gci_path = results_folder + '1000shots_gci_results.json'
with open(vqe_path, 'r') as f:
    gci_results = json.load(f)

vqe_path_noise = results_folder + '1000shots_vqe_results_noise.json'
with open(vqe_path, 'r') as f:
    vqe_results_noise = json.load(f)

gci_path_noise = results_folder + '1000shots_gci_results_noise.json'
with open(vqe_path, 'r') as f:
    gci_results_noise = json.load(f)

In [ ]:
vqe_expectation = vqe_results['total_expectation_energy']
gci_expectation = gci_results['total_expectation_energy']
vqe_expectation_noise = vqe_results_noise['total_expectation_energy']
gci_expectation_noise = gci_results_noise['total_expectation_energy']

-6.320729216


## Report

In [41]:
report = report_ionq(expectation_value_vqe, expectation_value_gci, ham_matrix, vqe_expectation, gci_expectation, vqe_expectation_noise, gci_expectation_noise)

In [42]:
df = report_table(report)
df

NameError: name 'report_table' is not defined